In [12]:
# %load 1_微博爬虫v2.py
# 程序功能: 按关键字爬取微博清单
# 程序作者: 马哥python说
import os
import re  # 正则表达式提取文本
from jsonpath import jsonpath  # 解析json数据
import requests  # 发送请求
import pandas as pd  # 存取csv文件
import datetime  # 转换时间用

# 请求头
headers = {
	"User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Mobile Safari/537.36",
	"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
	"accept-encoding": "gzip, deflate, br",
}


def trans_time(v_str):
	"""转换GMT时间为标准格式"""
	GMT_FORMAT = '%a %b %d %H:%M:%S +0800 %Y'
	timeArray = datetime.datetime.strptime(v_str, GMT_FORMAT)
	ret_time = timeArray.strftime("%Y-%m-%d %H:%M:%S")
	return ret_time


def getLongText(v_id):
	"""爬取长微博全文"""
	url = 'https://m.weibo.cn/statuses/extend?id=' + str(v_id)
	r = requests.get(url, headers=headers)
	json_data = r.json()
	long_text = json_data['data']['longTextContent']
	# 微博内容-正则表达式数据清洗
	dr = re.compile(r'<[^>]+>', re.S)
	long_text2 = dr.sub('', long_text)
	# print(long_text2)
	return long_text2


def get_weibo_list(v_keyword, v_max_page):
	"""
	爬取微博内容列表
	:param v_keyword: 搜索关键字
	:param v_max_page: 爬取前几页
	:return: None
	"""
	for page in range(1, v_max_page + 1):
		print('===开始爬取第{}页微博==='.format(page))
		# 请求地址
		url = 'https://m.weibo.cn/api/container/getIndex'
		# 请求参数
		params = {
			"containerid": "100103type=1&q={}".format(v_keyword),
			"page_type": "searchall",
			"page": page
		}
		# 发送请求
		r = requests.get(url, headers=headers, params=params)
		print(r.status_code)
		# pprint(r.json())
		# 解析json数据
		cards = r.json()["data"]["cards"]
		# 微博内容
		text_list = jsonpath(cards, '$..mblog.text')
		# 微博内容-正则表达式数据清洗
		dr = re.compile(r'<[^>]+>', re.S)
		text2_list = []
		print('text_list is:')
		print(text_list)
		if not text_list:  # 如果未获取到微博内容，进入下一轮循环
			continue
		if type(text_list) == list and len(text_list) > 0:
			for text in text_list:
				text2 = dr.sub('', text)  # 正则表达式提取微博内容
				print(text2)
				text2_list.append(text2)
		# 微博创建时间
		time_list = jsonpath(cards, '$..mblog.created_at')
		time_list = [trans_time(v_str=i) for i in time_list]
		# 微博作者
		author_list = jsonpath(cards, '$..mblog.user.screen_name')
		# 微博id
		id_list = jsonpath(cards, '$..mblog.id')
		 #认证
		verified=jsonpath(cards, '$..mblog.verified')
		# 判断是否存在全文
		isLongText_list = jsonpath(cards, '$..mblog.isLongText')
		idx = 0
		for i in isLongText_list:
			if i == True:
				long_text = getLongText(v_id=id_list[idx])
				text2_list[idx] = long_text
			idx += 1
		# 转发数
		reposts_count_list = jsonpath(cards, '$..mblog.reposts_count')
		# 评论数
		comments_count_list = jsonpath(cards, '$..mblog.comments_count')
		# 点赞数
		attitudes_count_list = jsonpath(cards, '$..mblog.attitudes_count')
		# 把列表数据保存成DataFrame数据
		df = pd.DataFrame(
			{
				'页码': [page] * len(id_list),
				'微博id': id_list,
				'微博作者': author_list,
				'是否认证':verified,
				'发布时间': time_list,
				'微博内容': text2_list,
				'转发数': reposts_count_list,
				'评论数': comments_count_list,
				'点赞数': attitudes_count_list,
			}
		)
		# 表头
		if os.path.exists(v_weibo_file):
			header = None
		else:
			header = ['页码', '微博id', '微博作者', '是否认证','发布时间', '微博内容', '转发数', '评论数', '点赞数']  # csv文件头
		# 保存到csv文件
		df.to_csv(v_weibo_file, mode='a+', index=False, header=header, encoding='utf_8_sig')
		print('csv保存成功:{}'.format(v_weibo_file))


if __name__ == '__main__':
	# 爬取前几页
	max_search_page = 49  # 爬前n页
	# 爬取关键字
	search_keyword = '台风梅花'
	# 保存文件名
	v_weibo_file = '微博清单_{}_前{}页.csv'.format(search_keyword, max_search_page)
	# 如果csv文件存在，先删除之
	if os.path.exists(v_weibo_file):
		os.remove(v_weibo_file)
		print('微博清单存在，已删除: {}'.format(v_weibo_file))
	# 调用爬取微博函数
	get_weibo_list(v_keyword=search_keyword, v_max_page=max_search_page)
	# 数据清洗-去重
	df = pd.read_csv(v_weibo_file)
	# 删除重复数据
	df.drop_duplicates(subset=['微博id'], inplace=True, keep='first')
	# 再次保存csv文件
	df.to_csv(v_weibo_file, index=False, encoding='utf_8_sig')
	print('数据清洗完成')


===开始爬取第1页微博===
200
text_list is:
['【<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E7%9B%B4%E6%92%AD%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1%23&extparam=%23%E7%9B%B4%E6%92%AD%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1%23&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#直播台风梅花#</span></a>】今天，中央气象台发布<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%232022%E9%A6%96%E4%B8%AA%E5%8F%B0%E9%A3%8E%E7%BA%A2%E8%89%B2%E9%A2%84%E8%AD%A6%23&extparam=%232022%E9%A6%96%E4%B8%AA%E5%8F%B0%E9%A3%8E%E7%BA%A2%E8%89%B2%E9%A2%84%E8%AD%A6%23&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#2022首个台风红色预警#</span></a>，预计台风“梅花”即将于今天傍晚前后在浙江三门到舟山一带沿海登陆（强台风级，42-48米/秒，14-15级），并将于今天夜间在浙江嘉兴到上海浦东一带沿海再次登陆。江浙沪等地将迎来强风暴雨。最新情况如何？关注！  <a  href="https://weibo.com/l/wblive/p/show/1022:2321324813563813560375" data-hide=""><span class=\'url-

csv保存成功:微博清单_台风梅花_前49页.csv
===开始爬取第2页微博===
200
text_list is:
['<a  href="https://m.weibo.cn/p/index?extparam=%E7%AB%B9%E5%BD%B1%E7%A6%85%E9%A3%8E&containerid=100808b595b4fee1dc87f8a2612fcdd7d0eb8a&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">竹影禅风</span></a><br />【七律              台风梅花过后赏荷】<br /><br />寸心惟欲人间住，叵奈浮生多鹢路。<br />初蕊无衰溽暑风，仲秋却损梅花雨。<br />曾经碧叶可拿云，更使骚人长驻足。<br />今夕余青照水寒，重来再把清香吐。 ', '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E4%B8%8D%E9%80%A0%E8%B0%A3%E4%B8%8D%E4%BC%A0%E8%B0%A3%E4%BA%BA%E4%BA%BA%E6%9C%89%E8%B4%A3%23&extparam=%23%E4%B8%8D%E9%80%A0%E8%B0%A3%E4%B8%8D%E4%BC%A0%E8%B0%A3%E4%BA%BA%E4%BA%BA%E6%9C%89%E8%B4%A3%23&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><

csv保存成功:微博清单_台风梅花_前49页.csv
===开始爬取第4页微博===
200
text_list is:
['九月plog是<br />-与家里人共进晚餐<br />-跟方先生挖掘新店吃吃喝喝<br />-在梅山经历台风🌀“梅花”<br />-🎑中秋节宁大学子特有的月饼 ', '【最新消息！<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%AE%81%E6%B3%A2%E8%BF%99%E7%89%87%E7%BD%91%E7%BA%A2%E7%B2%89%E9%BB%9B%E8%8A%B1%E6%B5%B7%E6%9A%82%E4%B8%8D%E5%AF%B9%E5%A4%96%E5%BC%80%E6%94%BE%23&extparam=%23%E5%AE%81%E6%B3%A2%E8%BF%99%E7%89%87%E7%BD%91%E7%BA%A2%E7%B2%89%E9%BB%9B%E8%8A%B1%E6%B5%B7%E6%9A%82%E4%B8%8D%E5%AF%B9%E5%A4%96%E5%BC%80%E6%94%BE%23&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#宁波这片网红粉黛花海暂不对外开放#</span></a><span class="url-icon"><img alt=[话筒] src="https://h5.sinaimg.cn/m/emoticon/icon/others/o_huatong-0a3e80b147.png" style="width:1em; height:1em;" /></span>】刚刚，一则关于鄞州区塘溪镇的“网红花海”——粉黛乱子草暂时关闭的消息，在宁波人朋友圈不胫而走。从塘溪镇旅游办获悉，该镇确实于今日发出了相关公告，“粉黛花海暂不对外开放”，究其原因是：“因前阵子的台风——‘梅花’，导致部分进山道 ...<a href="/status/4818751533092635">全文</

csv保存成功:微博清单_台风梅花_前49页.csv
===开始爬取第6页微博===
200
text_list is:
['大家好，我是嘉兴的孩子。我的家乡正在遭受台风“梅花”的袭击，供水和食物供给都是问题，今晚台风过境，明天估计只有肯德基能正常营业。v我50，助我星期四顺利抗台 ', '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%B0%B4%E5%8A%A1%E5%8A%A8%E6%80%81%23&isnewpage=1&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#水务动态#</span></a> 近日，市秦淮河河道管理处对河道堤防全线进行检查，全面排查第12号台风“梅花”带来的影响，消除河道安全隐患。（来源：市秦淮河河道管理处  审核：李明阳、马宁 发布：孙雪） ', '好不容易回趟家🏰，居然挂那么大的台风🌬（9.14梅花真的好强）<br /><br /><a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E4%B8%8A%E6%B5%B7%E8%BF%AA%E5%A3%AB%E5%B0%BC%23&isnewpage=1&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#上海迪士尼#</span></a><a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9D%B0%E6%8B%89%E5%A4%9A%E5%B0%BC%23&isnewpage=1&luicode=10000011&lfid=100103type%3D1%26q%3

csv保存成功:微博清单_台风梅花_前49页.csv
===开始爬取第8页微博===
200
text_list is:
['【每日期货市场要闻速递（9月26日）】<br /><br />1. 据中国港口协会数据，9月中旬，我会监测沿海重点枢纽港口货物吞吐量同比增加8.8%。其中外贸同比增加4.2%，内贸同比增加13.13%。长江枢纽港口吞吐量同比增加3.9%，集装箱同比减少13.8%。9月中旬，台风“梅花”对港口生产带来不同程度影响，有的港口一度停工， ...<a href="/status/4817874801659691">全文</a>', '台风梅花走后，天气忽然就变得凉爽了许多，适逢周末，秋高气爽，今早醒来，微信朋友圈都是昨天秋游的的记录，看着真好！ ', '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E7%A4%BE%E4%BC%9A%E7%99%BE%E6%80%81%23&isnewpage=1&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#社会百态#</span></a><a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E8%BD%BB%E6%9D%BE%E4%B8%80%E5%88%BB%23&isnewpage=1&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#轻松一刻#</span></a><br /><br />台风“梅花”即将登陆。登陆前风力有多大？网友泼了一桶水……视频来源 金蛇郎君 <a  href="https://video.weibo.com/show?fid=1034:4817376894713990" data-hid

csv保存成功:微博清单_台风梅花_前49页.csv
===开始爬取第10页微博===
200
text_list is:
['<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1%E7%9A%84%E5%BD%B1%E5%93%8D%23&extparam=%23%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1%E7%9A%84%E5%BD%B1%E5%93%8D%23&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#台风梅花的影响#</span></a>【今天全部修复了】<br />台风梅花的肆虐，吹倒了修理厂大门，经过几天的努力，今天终于完成了修复，可以睡个安稳觉了，连续几天的担心和工作，感觉身体很累，明天好好休息一天。 ', '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%95%B0%E5%AD%97%E5%8C%97%E4%BB%91%23&extparam=%23%E6%95%B0%E5%AD%97%E5%8C%97%E4%BB%91%23&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#数字北仑#</span></a>【数字人民币应用有新突破 北仑区首笔数字人民币保险赔付场景正式落地】受此次台风“梅花”影响，区内部分农业种植户受到不同程度的损失，灾情发生后，区地方金融监管局积极履职，主动担当，引导区内各保险机构及时帮助受灾农户解决资金问题。9月下旬，太平洋产险北仑中心支公司首次通过数字 ...<a href="/status/481721662806791

csv保存成功:微博清单_台风梅花_前49页.csv
===开始爬取第12页微博===
200
text_list is:
['🍀舟山行枸杞岛日记<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%85%E8%A1%8C&containerid=10080887483408cfa62006f530473e57323a26&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">旅行</span></a> <br />被困在岛上足足5天<br />正面迎接了梅花15级台风<br />在岛上的小酒吧结识了超多伙伴<br />每个人都很特别，很有故事<br />还陪新朋友和老友度过了超有意思的生日<br />谁能说这场旅行不是一场刺激又难忘的冒险呢？<br /><a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%97%85%E8%A1%8C%23&isnewpage=1&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8F%B0%E9%A3%8E%E6%A2%85%E8%8A%B1" data-hide=""><span class="surl-text">#旅行#</span></a> <a  href="http://weibo.com/p/100101B2094452D265A7FE4793" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=

csv保存成功:微博清单_台风梅花_前49页.csv
===开始爬取第14页微博===
200
text_list is:
['实话实说；我们家乡数月无雨下。原因是因为家乡的某些人们做了祸害人类的事情。所以上天不肯降雨。数月前我看见庄稼要被旱死，所以我的善意触发，我为家乡乡们借来两次短时间暴雨缓解了一下旱情，现在我受伤了无法施展借雨。前几天因台风梅花带来了一点毛毛雨连地皮都没有下湿。如果我没有受伤，这次我 ...<a href="/status/4816419604661699">全文</a>', '成长纪要：2022.09，台风“梅花”到访，儿子半夜起床关客厅和餐厅的门窗，抢在老公之前妥妥办好 ', '那几天台风梅花要来，声势浩大，我想到的却是，小花梅。 ', '台风梅花，来过的痕迹。 ', '9月份中美航司被“熔断”，中美直航几乎已是不可能，只能选择了国泰PVG/HKG/BOS，各种不确定因素，甚至一场梅花台风🌀，实在是让人捉摸不透与莫名上火，结果还是能够十分“丝滑”的出入境，如愿以偿的开启一段在大波村常态化家庭团聚…… <a  href="http://weibo.com/p/100101B2094655D26EA2FB479F" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://h5.sinaimg.cn/upload/2015/09/25/3/timeline_card_small_location_default.png\'></span><span class="surl-text">上海·上海浦东国际机场2号航站楼</span></a> ', '记录下最近几天<br /><br />1.花了很多时间看SUV，各种问题，放弃了。<br />2.回到原点，买轿车，综合考虑，直奔凯美瑞。<br />3.今天卖车，明天各家报价，争取周五出手。<br />4.要去南京公司了，这两个月出来溜了一圈。车子用得很少，这次换车，主要是出于安全考虑。<br /><br />梅花台风，老车子起雾，后视镜基本看不到，空调系统也不行 ...<a href="/status/4816269582274668">全文</a>', '【你知道吗？原来

PermissionError: [Errno 13] Permission denied: '微博清单_台风梅花_前49页.csv'

In [ ]:
print()